# ANLI - GloVE

https://towardsdatascience.com/glove-elmo-bert-9dbbc9226934

In [1]:
# Only works with jdk8. Be sure to switch jdk version to 8 for this notebook to run.
!java -version

java version "1.8.0_361"
Java(TM) SE Runtime Environment (build 1.8.0_361-b09)
Java HotSpot(TM) 64-Bit Server VM (build 25.361-b09, mixed mode)


In [2]:
!python3 -m venv .sparknlp-env

In [3]:
!source .sparknlp-env/bin/activate

In [4]:
pip install -q torch

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install -q spark-nlp==4.4.1

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install -q pyspark==3.3.1

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install -U pandas==1.5.3

Note: you may need to restart the kernel to use updated packages.


In [8]:
from datasets import load_dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from torchtext.vocab import GloVe
import pandas as pd

In [9]:
import pyspark
import sparknlp

In [10]:
# load datasets and convert them into dataframes
dataset = load_dataset('anli')
train_r1 = pd.DataFrame(dataset['train_r1'])
dev_r1 = pd.DataFrame(dataset['dev_r1'])
test_r1 = pd.DataFrame(dataset['test_r1'])
train_r2 = pd.DataFrame(dataset['train_r1'])
dev_r2 = pd.DataFrame(dataset['dev_r2'])
test_r2 = pd.DataFrame(dataset['test_r2'])
train_r3 = pd.DataFrame(dataset['train_r3'])
dev_r3 = pd.DataFrame(dataset['dev_r3'])
test_r3 = pd.DataFrame(dataset['test_r3'])

Found cached dataset anli (/Users/sofiamurillosanchez/.cache/huggingface/datasets/anli/plain_text/0.1.0/aabce88453b06dff21c201855ea83283bab0390bff746deadb30b65695755c0b)


  0%|          | 0/9 [00:00<?, ?it/s]

In [11]:
def convert_to_glove(df):
    new_df = {'label': df['label'], 'text': df['hypothesis'] + df['premise']}
    return pd.DataFrame(data=new_df)

In [12]:
glove_train_r1 = convert_to_glove(train_r1)
glove_dev_r1 = convert_to_glove(dev_r1)
glove_test_r1 = convert_to_glove(test_r1)

glove_train_r2 = convert_to_glove(train_r2)
glove_dev_r2 = convert_to_glove(dev_r2)
glove_test_r2 = convert_to_glove(test_r2)

glove_train_r3 = convert_to_glove(train_r3)
glove_dev_r3 = convert_to_glove(dev_r3)
glove_test_r3 = convert_to_glove(test_r3)
glove_train_r1

,label,text
0,0,The trolleybus system has over 2 urban routesT...
1,1,Sharron Macready was a popular character throu...
2,1,Bastedo didn't keep any pets because of her vi...
3,1,Alexandra Bastedo was named by her mother.Alex...
4,1,Bastedo cared for all the animals that inhabit...
...,...,...
16941,2,The KNVB cup trophy was awarded to Heerenveen....
16942,2,20th Century Fox Television is the syndication...
16943,2,Olivier Rochus always plays with a partner.Oli...
16944,2,Liisi Rist (born 26 June 1991) is an Estonian ...


In [ ]:
spark = sparknlp.start()

23/05/03 13:24:42 WARN Utils: Your hostname, Sofias-MacBook-Pro-2.local resolves to a loopback address: 127.0.0.1; using 10.8.146.230 instead (on interface en0)
23/05/03 13:24:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/Users/sofiamurillosanchez/opt/anaconda3/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/sofiamurillosanchez/.ivy2/cache
The jars for the packages stored in: /Users/sofiamurillosanchez/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8e682cfd-ba55-45a9-998a-03ac5c4a8a71;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;4.4.1 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1.16.8 in central
	found com.google.cloud#google-cloud-storage;2.16.0 in central
	found com.google.guava#guava;31.1-jre in central
	f

23/05/03 13:24:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [ ]:
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
import pandas as pd

In [ ]:
document_assembler = DocumentAssembler() \
.setInputCol("text") \
.setOutputCol("document") \
.setCleanupMode("shrink")
    
tokenizer = Tokenizer() \
.setInputCols(["document"]) \
.setOutputCol("token") \
.setSplitChars(['-']) \
.setContextChars(['(', ')', '?', '!', '#', '@']) 

normalizer = Normalizer() \
.setInputCols(["token"]) \
.setOutputCol("normalized")\
.setCleanupPatterns(["[^\w\d\s]"]) 

stopwords_cleaner = StopWordsCleaner()\
.setInputCols("normalized")\
.setOutputCol("cleanTokens")\
.setCaseSensitive(False)

lemma = LemmatizerModel.pretrained('lemma_antbnc') \
.setInputCols(["cleanTokens"]) \
.setOutputCol("lemma")

glove_embeddings = WordEmbeddingsModel().pretrained() \
.setInputCols(["document",'lemma'])\
.setOutputCol("embeddings")\
.setCaseSensitive(False)

embeddingsSentence = SentenceEmbeddings() \
.setInputCols(["document", "embeddings"]) \
.setOutputCol("sentence_embeddings") \
.setPoolingStrategy("AVERAGE")

classsifierdl = ClassifierDLApproach()\
.setInputCols(["sentence_embeddings"])\
.setOutputCol("class")\
.setLabelColumn("label")\
.setMaxEpochs(5)\
.setLr(0.001)\
.setBatchSize(8)\
.setEnableOutputLogs(True)
#.setOutputLogsPath('logs')

glove_clf_pipeline = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            lemma, 
            glove_embeddings,
            embeddingsSentence,
            classsifierdl])

In [ ]:
train_1 = glove_train_r1.dropna(subset=['text', 'label'])
sparkDF_1=spark.createDataFrame(train_1) 
train_2 = glove_train_r2.dropna(subset=['text', 'label'])
sparkDF_2=spark.createDataFrame(train_2) 
train_3 = glove_train_r3.dropna(subset=['text', 'label'])
sparkDF_3=spark.createDataFrame(train_3)

test_1 = glove_test_r1.dropna(subset=['text', 'label'])
sparkDF_test1=spark.createDataFrame(test_1)
test_2 = glove_test_r2.dropna(subset=['text', 'label'])
sparkDF_test2=spark.createDataFrame(test_2) 
test_3 = glove_test_r3.dropna(subset=['text', 'label'])
sparkDF_test3=spark.createDataFrame(test_3) 

glove_clf_pipelineModel = glove_clf_pipeline.fit(sparkDF_1)
glove_clf_pipelineModel = glove_clf_pipeline.fit(sparkDF_2)
glove_clf_pipelineModel = glove_clf_pipeline.fit(sparkDF_3)

In [ ]:
preds = glove_clf_pipelineModel.transform(sparkDF_test1)
preds += glove_clf_pipelineModel.transform(sparkDF_test2)
preds += glove_clf_pipelineModel.transform(sparkDF_test3)
preds.select('text','target',"class.result").show(10, truncate=80)

In [ ]:
# model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self

# # If we save using the predefined names, we can load using `from_pretrained`
# output_model_file = os.path.join(OUTPUT_DIR, WEIGHTS_NAME)
# output_config_file = os.path.join(OUTPUT_DIR, CONFIG_NAME)

# torch.save(model_to_save.state_dict(), output_model_file)
# model_to_save.config.to_json_file(output_config_file)
# tokenizer.save_vocabulary(OUTPUT_DIR)

https://pytorch.org/docs/stable/nn.html#torch.nn.Module

https://huggingface.co/docs/transformers/model_doc/bert

https://huggingface.co/docs/transformers/v4.28.1/en/main_classes/model#transformers.PreTrainedModel

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ef800559-4ff5-4f9a-a563-f7fcfd96a58b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>